<h1><center>IMT2118 - Ciencia de Datos Geoespaciales (2024-1) </center></h1>
<h2><center>Tarea 4:  Detección Automática de Cultivos en Egipto. </center></h2>



<div class="alert alert-block alert-info">

## Instrucciones generales.

&#128467; **Fecha de entrega:** 23 de junio 2024, 23:59. <br>

&#128236; **Entrega:** a través de Canvas, adjunte todos los archivos necesarios para ejecutar su código dentro de una única carpeta comprimida (.zip). Incluya el  Jupyter Notebook con el desarrollo (puede utilizar este mismo notebook), y las capas o archivos vectoriales que haya generado como parte de su análisis. El objetivo es que **el Notebook pueda correr completamente dentro de la carpeta entregada**. <br>

&#128013; La Tarea debe ser desarrollada en lenguaje de programación Python. <br>

&#11088; En el Notebook, incluya texto explicando los pasos u operaciones desarrolladas, no sólo el código. El objetivo es que el Notebook siga un hilo narrativo, sea claro y atractivo de leer. En [este link](https://github.com/brad-do/nb-miscellany/blob/master/things_to_do_in_markdown.ipynb) puede encontrar algunos tips útiles para usar celdas de anotaciones o Markdown. <br>


&#128519; El trabajo es **invididual**, y debe seguir todos los lineamientos de integridad académica del curso. Recuerde referenciar apropiadamente todas las referencias, códigos y datos consultados para el desarrollo de su tarea.<br>

</div>

<div class="alert alert-block alert-info">

### Nota: partes de esta tarea serán desarrolladas progresivamente en clases y controles. Se recomienda fuertemente asistir a clases con su computador, de manera de resolver dudas y avanzar en el desarrollo de la misma.
</div>

## 1. Introducción.

En esta tarea, desarrollaremos el flujo completo de extracción automatizada de una máscara de cultivos (*crops*) para Egipto, mediante algoritmos de segmentación y clasificación supervisada. Este ejercicio está inpirado en el este [caso de estudio](https://docs.digitalearthafrica.org/en/latest/sandbox/notebooks/Real_world_examples/Scalable_machine_learning/0_README.html) desarrollado por Digital Earth Africa.

Utilizaremos un conjunto de datos de entrenamiento de Egipto, que contiene etiquetas de "cultivo" (*crop*) y "no cultivo" (*non-crop*), etiquetadas como 1 y 0, respectivamente.  Al final de esta Tarea, habremos producido un modelo para identificar áreas de tierras de cultivo  y generaremos una máscara de tierras de cultivo (como un geotiff) para la región alrededor de la ciudad de Faiyum, Egipto.

## 2. Datos.

Los datos de entrenamiento se denominan `crop_training_egypt.geojson`. 

## 3. Desarrollo.

#### 3.1. Lea y grafique los datos de entrenamiento entregados 
#### 3.2. Defina un área de interés (AOI) que comprenda todos los puntos de entrenamiento.
#### 3.3 Seleccione imágenes de la colección "COPERNICUS/S2_SR_HARMONIZED" sobre la región de estudio para el año 2019. Aplique las máscaras, correcciones y escalamientos adecuados a estas imágenes.
#### 3.4. Para cada una de las imágenes seleccionadas, agregue como bandas adicionales los siguientes índices: NDVI, LAI (leaf area index), MNDWI (modified normalized difference water index).
#### 3.5. Obtenga la mediana de la colección de imágenes. De esta forma, obtendrá una imagen multibanda representativa del año 2019. 

Genere una visualización RGB de la mediana como forma de verificación.

#### 3.6. Utilizando los datos de entrenamiento, obtenga estadísticas zonales para cada uno de los polígonos etiquetados. Es decir, calcule el valor promedio de las siguientes bandas: B2-B7, B11, B12, NIR, LAI, MNDWI. De esta forma, su dataset de entrenamiento tendrá ahora 12 nuevas columnas. 

Se sugiere user *Reducers* de GEE, de manera de no tener que exportar las imágenes y realizar todos los cálculos en la nube.

#### 3.7 Entrene un modelo de clasificación binaria (por ejemplo, kNN o RandomForest), justificando su selección de hiperparámetros.
Para optimizar los parámetros del modelo, se sugiere usar [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) para buscar exhaustivamente en un conjunto de parámetros y determinar la mejor combinación. Esto dará como resultado el mejor rendimiento del modelo según la métrica de precisión definida.

#### 3.8 Evalúe el rendimiento del clasificador utilizando métricas como accuracy, F1 y ROC.

Una vez que haya seleccionado el modelo con mejor rendimiento, guarde el modelo entrenado para aplicarlo a nuevas imágenes.

#### 3.9 Aplique el clasificador entrenado en el punto anterio para generar nuevas predicciones en las ubicaciones de prueba definidas más abajo (`testing_locations`)

Para aplica el modelo de clasificación, siga los siguientes pasos:

- Seleccione una sub-imagen centrada en la ubicación de prueba. *Sugerencia*: Comience con una sección pequeña, y expanda a de acuerdo la capcidad de cómputo de su sistema.
- Aplique un algoritmo de segmentación (por ejemplo, SNIC) sobre este segmento de imagen. Justifique su selección de parámetros y genere una visualización de los segmentos.

- Para cada segmento, calcule las estadísticas zonales indicadas en el punto 3.6. De esta forma, ud. podrá generar un dataframe con la siguiente estructura:

| segmentID | B2 | B3 |...|B7 |B11|B12|NDVI|MNDWI| LAI|
|---| --- | --- |--- | --- | --- | --- | --- |--- | --- |
| 1 | ... | ... ... | ...| ... | ... | ... ... | ...|
| ...| ... | ... ... | ...| ... | ... | ... ... | ...|
| n | ... | ... ... | ...| ... | ... | ... ... | ...|

Donde $n$ es el número de segmentos en su imagen.

- Aplique el clasificador sobre este dataframe de prueba, y genere las predicciones de clase (0/1) para cada segmento.
- Genere una visualización del resultado de la clasificación para la imagen de prueba, y comente sobre los resultados.



In [105]:
from shapely.geometry import mapping
import pandas as pd
import geopandas as gpd
import numpy as np
import geemap, ee
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import branca
import os
import seaborn as sns
import pandas as pd
import folium
import json
import fiona
from IPython.display import display
from shapely.geometry import MultiPolygon, Polygon
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


fiona.drvsupport.supported_drivers['KML'] = 'r'

### Parametros de visualización

In [106]:
testing_locations = {
    '1': (24.4780, 33.0457),
    '2': (22.7712, 28.5223),
    '3': (30.3971, 30.7996),
    '4': (31.1751, 31.2221),
}

# Paleta de colores para MNDWI con valores entre -1 y 1
mndwi_palette = [
    '#ffffff',  # -1.0 < MNDWI ≤ -0.8 (blanco)
    '#d9d9d9',  # -0.8 < MNDWI ≤ -0.6 (gris claro)
    '#b3b3b3',  # -0.6 < MNDWI ≤ -0.4 (gris)
    '#8c8c8c',  # -0.4 < MNDWI ≤ -0.2 (gris oscuro)
    '#666666',  # -0.2 < MNDWI ≤ 0.0 (gris más oscuro)
    '#3399ff',  # 0.0 < MNDWI ≤ 0.2 (azul claro)
    '#0073e6',  # 0.2 < MNDWI ≤ 0.4 (azul)
    '#0059b3',  # 0.4 < MNDWI ≤ 0.6 (azul oscuro)
    '#003d80',  # 0.6 < MNDWI ≤ 0.8 (azul más oscuro)
    '#00264d'   # 0.8 < MNDWI ≤ 1.0 (azul muy oscuro)
]

# Parámetros de visualización para MNDWI
mndwiParams = {
    'min': -1.0,
    'max': 1.0,
    'palette': mndwi_palette
}


visualization = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 0.3}

palett = [
    '#800000',  # -0.05 < LAI ≤ -0.045
    '#b22222',  # -0.045 < LAI ≤ -0.04
    '#ff4500',  # -0.04 < LAI ≤ -0.035
    '#ff6347',  # -0.035 < LAI ≤ -0.03
    '#ff7f50',  # -0.03 < LAI ≤ -0.025
    '#ff8c00',  # -0.025 < LAI ≤ -0.02
    '#ffa500',  # -0.02 < LAI ≤ -0.015
    '#ffb732',  # -0.015 < LAI ≤ -0.01
    '#ffc758',  # -0.01 < LAI ≤ -0.005
    '#ffd87a',  # -0.005 < LAI ≤ 0.0
    '#ffeb8c',  # 0.0 < LAI ≤ 0.05
    '#ffffbf',  # 0.05 < LAI ≤ 0.1
    '#d9ef8b',  # 0.1 < LAI ≤ 0.2
    '#a6d96a',  # 0.2 < LAI ≤ 0.3
    '#66bd63',  # 0.3 < LAI ≤ 0.4
    '#1a9850',  # 0.4 < LAI ≤ 0.5
    '#006837',  # 0.5 < LAI ≤ 0.6
    '#004529',  # 0.6 < LAI ≤ 0.7
    '#003322',  # 0.7 < LAI ≤ 0.8
    '#002218',  # 0.8 < LAI ≤ 0.9
    '#001110',  # 0.9 < LAI ≤ 1.0
    '#000000',  # 1.0 < LAI ≤ 1.2
    '#001110',  # 1.2 < LAI ≤ 1.4
    '#002218',  # 1.4 < LAI ≤ 1.6
    '#003322',  # 1.6 < LAI ≤ 1.8
    '#004529',  # 1.8 < LAI ≤ 2.0
    '#006837',  # 2.0 < LAI ≤ 2.5
    '#1a9850',  # 2.5 < LAI ≤ 3.0
    '#66bd63',  # 3.0 < LAI ≤ 3.5
]

# Parámetros de visualización para LAI
laiParams = {
    'min': -0.05,
    'max': 3.5,
    'palette': palett
}


ndvi_palette = [
    '#000000',
    '#a50026',
    '#d73027',
    '#f46d43',
    '#fdae61',
    '#fee08b',
    '#ffffbf',
    '#d9ef8b',
    '#a6d96a',
    '#66bd63',
    '#1a9850',
    '#006837'
]

# Parámetros de visualización para NDVI
ndviParams = {
    'min': -1.0,
    'max': 1.0,
    'palette': ndvi_palette
}

legend_css = """
<style>
    .leaflet-control .leaflet-bar {
        bottom: 0;
        top: auto;
    }
    .legend {
        background-color: white;
        bottom: 20px;
        left: 50%;
        transform: translateX(-50%);
        padding: 10px;
        position: absolute;
        z-index: 9999;
        box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
        border: 1px solid black;
    }
    .legend .colorbar {
        border: 1px solid black;
    }
</style>
"""

# Parametros de Folium

# Codigo proporcionado por el copilot chat de github:

# Función para añadir capas Earth Engine
def add_ee_layer(fmap, image, vis_params, name):
    map_id_dict = ee.Image(image).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(fmap)

# Función para crear y añadir leyendas de colores con CSS personalizado
def add_color_map(fmap, palette, vmin, vmax, caption, legend_css):
    color_map = branca.colormap.LinearColormap(palette, vmin=vmin, vmax=vmax).to_step(n=len(palette))
    color_map.caption = caption
    fmap.get_root().html.add_child(folium.Element(legend_css + color_map._repr_html_().replace('<div', '<div class="legend"')))

folium.Map.add_ee_layer = add_ee_layer

In [107]:
ee.Authenticate()
ee.Initialize(project='ee-ortiznicolas')

### Datos.

Los datos de entrenamiento se denominan `crop_training_egypt.geojson`. 


In [108]:
# Load input data shapefile
data = gpd.read_file(os.path.join('data', 'crop_training_egypt.geojson'))

In [109]:
def mask_s2_clouds(image):

  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0)))

  return image.updateMask(mask).divide(10000)

### Zona de Investigación (AOI)

In [110]:
# definir el aoi en base a mis puntos de entrenamiento

gdf_aoi = gpd.read_file(os.path.join('data', 'aoi.kml'), driver='KML')
aoi = ee.Geometry.Polygon(np.array(list(gdf_aoi.geometry.unary_union.exterior.coords))[:, 0:2].tolist())

In [111]:
fmap = folium.Map(location=aoi.centroid().coordinates().getInfo()[::-1], zoom_start=4)
folium.GeoJson(gdf_aoi.geometry.to_json()).add_to(fmap)
display(fmap)

### Imagenes Satelitales de `COPERNICUS/S2_SR_HARMONIZED`

Definimos las funciones para calcular los indices espectrales.

In [112]:
# Extraido: https://www.lifeingis.com/computation-of-mndwi-in-google-earth-engine-using-sentinel-2/

def get_MNDWI(image):
    mndwi = image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
    return image.addBands(mndwi)

def get_NDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Extraido: https://kaflekrishna.com.np/blog-detail/enhanced-vegetation-index-evi-sentinel-2-image-google-earth-engine/

def getEVI(image):
    # Compute the EVI using an expression.
    EVI = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('B8').divide(10000),
            'RED': image.select('B4').divide(10000),
            'BLUE': image.select('B2').divide(10000)
        }).rename("EVI")

    image = image.addBands(EVI)

    return(image)

# Extraido: https://kaflekrishna.com.np/blog-detail/calculating-different-vegetation-indices-google-earth-engine-sentinel-2-images/

def get_LAI(image):
    image = getEVI(image)

    LAI = image.expression(
        '(3.618*EVI - 0.118)', {
            'EVI': image.select('EVI')
        }).rename("LAI")
    return image.addBands(LAI)

### Descarga de las imagenes para el año 2019


Donde se le aplican las correcciones y escalados necesarios a las imagenes, incluso se calculan los indices espectrales solicitados y se obtiene la mediana de la coleccion de imagenes.

In [113]:
# COPERNICUS/S2_SR_HARMONIZED

s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(aoi).filterDate('2019-01-01', '2019-12-31')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).map(mask_s2_clouds)
    .map(get_MNDWI).map(get_NDVI).map(get_LAI).median())

### Mapa de NDVI, LAI y MNDWI

In [114]:
fmap = folium.Map(location=aoi.centroid().coordinates().getInfo()[::-1], zoom_start=4)

# Añadir capas
add_ee_layer(fmap, s2.select('MNDWI').clip(aoi), mndwiParams, 'MNDWI')
add_ee_layer(fmap, s2.select('LAI').clip(aoi), laiParams, 'LAI')
add_ee_layer(fmap, s2.select('NDVI').clip(aoi), ndviParams, 'NDVI')
add_ee_layer(fmap, s2.clip(aoi), visualization, 'RGB')

# Añadir leyendas de colores con CSS personalizado
add_color_map(fmap, ndvi_palette, -1, 1, 'NDVI\n', legend_css)
add_color_map(fmap, palett, laiParams["min"], laiParams["max"], 'LAI\n', legend_css)
add_color_map(fmap, mndwi_palette, -1, 1, 'MNDWI\n', legend_css)

# Añadir control de capas
fmap.add_child(folium.LayerControl())
fmap.save('map.html')
# Mostrar el mapa
display(fmap)

Si hay problemas con la visualizacion de las capas, existe un archivo html `map.html` que puede abrir en su navegador para visualizar los mapas.

### Estadisticas Zonales

In [115]:
# Sugerencia de chat-gpt

def geojson_to_ee(geojson):
    features = json.loads(geojson)['features']
    ee_features = []
    for feature in features:
        geom = ee.Geometry.Polygon(feature['geometry']['coordinates'])
        feat = ee.Feature(geom, feature['properties'])
        ee_features.append(feat)
    return ee.FeatureCollection(ee_features)

geojson = data.to_json()
training_polygons = geojson_to_ee(geojson)

bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B11', 'B12', "B8", 'MNDWI', 'LAI', 'NDVI']
stats = s2.select(bands).reduceRegions(
        collection=training_polygons,
        reducer=ee.Reducer.mean(),
        scale=10
    )

def extract_results(fc):
    features = fc.getInfo()['features']

    data = []
    for n, feature in enumerate(features):
        properties = feature['properties']
        data.append(properties)
        
    return pd.DataFrame(data)

df = extract_results(stats)

In [116]:

features = stats.getInfo()['features']
geometry = [features[n]['geometry']['coordinates'][0] for n in range(156)]
df['geometry'] = [MultiPolygon([Polygon(geometry[n])]) for n in range(156)]

gdf = gpd.GeoDataFrame(df, geometry='geometry')

In [117]:
gdf

B11       B12        B2        B3        B4        B5        B6  \
0    0.733292  0.678256  0.173934  0.288327  0.462798  0.510948  0.525262   
1    0.622793  0.586214  0.127251  0.221387  0.403995  0.450345  0.460985   
2    0.347494  0.300281  0.092490  0.130830  0.179501  0.198742  0.203350   
3    0.707838  0.669224  0.194167  0.316855  0.469252  0.515353  0.526149   
4    0.753900  0.692905  0.193576  0.309112  0.456322  0.502774  0.520972   
..        ...       ...       ...       ...       ...       ...       ...   
151  0.182459  0.109809  0.043651  0.071714  0.058178  0.111932  0.262828   
152  0.282790  0.198031  0.070715  0.104277  0.116745  0.165389  0.259204   
153  0.186612  0.109241  0.044571  0.076284  0.059926  0.114236  0.253208   
154  0.165859  0.084614  0.047793  0.078073  0.051289  0.114020  0.280206   
155  0.203549  0.123212  0.063145  0.092902  0.081158  0.137563  0.259407   

           B7        B8       LAI     MNDWI      NDVI  class  \
0    0.543644  0.555876 -0.117915 -0.437394  0.092134      0   
1    0.471448  0.474363 -0.117936 -0.475231  0.080267      0   
2    0.213668  0.207439 -0.117975 -0.464061  0.083398      0   
3    0.543404  0.547532 -0.117928 -0.381346  0.077993      0   
4    0.542706  0.548501 -0.117917 -0.419844  0.090711      0   
..        ...       ...       ...       ...       ...    ...   
151  0.305387  0.324972 -0.117766 -0.418258  0.669335      1   
152  0.291349  0.300595 -0.117843 -0.450953  0.429423      1   
153  0.302084  0.302299 -0.117786 -0.408565  0.647364      1   
154  0.354055  0.361438 -0.117727 -0.382589  0.736784      1   
155  0.297059  0.301056 -0.117808 -0.373931  0.561749      1   

                                              geometry  
0    MULTIPOLYGON (((26.19189 22.06193, 26.19230 22...  
1    MULTIPOLYGON (((32.24947 22.07338, 32.24989 22...  
2    MULTIPOLYGON (((32.62301 22.15862, 32.62342 22...  
3    MULTIPOLYGON (((28.35345 22.29337, 28.35386 22...  
4    MULTIPOLYGON (((27.72311 22.83994, 27.72352 22...  
..                                                 ...  
151  MULTIPOLYGON (((30.78142 31.17679, 30.78184 31...  
152  MULTIPOLYGON (((30.28155 30.48578, 30.28196 30...  
153  MULTIPOLYGON (((30.87481 30.88496, 30.87522 30...  
154  MULTIPOLYGON (((31.47082 30.96860, 31.47123 30...  
155  MULTIPOLYGON (((30.34884 30.97448, 30.34925 30...  

[156 rows x 14 columns]

### Algoritmo de Clasificación

In [130]:
X = gdf[['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B11', 'B12', "B8", 'MNDWI', 'LAI', 'NDVI']]
y = gdf['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=12)


# Hiperparametros para el modelo de Random Forest
param_grid = {
    'n_estimators': [50, 100, 200], # Numeros de arboles ha realizar
    'max_depth': [10, 20, 30, 40, 50, 80], # Profundidad maxima de los arboles
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True]
}

rf = RandomForestClassifier(random_state=42)
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)

grid_search_rf.fit(X_train, y_train)
y_pred = grid_search_rf.best_estimator_.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f'Accuracy: {accuracy} - F1 Score: {f1} - ROC AUC: {roc_auc} - Random Forest: {grid_search_rf.best_params_}')


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

knn = KNeighborsClassifier()
grid_search_knn = GridSearchCV(estimator=knn, param_grid={
    'n_neighbors': [3, 5, 7, 9, 11, 13, 20],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski', 'chebyshev']
}, cv=5, scoring='accuracy', verbose=1)

grid_search_knn.fit(X_train, y_train)
y_pred = grid_search_knn.best_estimator_.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

grid_search.fit(X_train, y_train)
y_pred = grid_search.best_estimator_.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f'Accuracy: {accuracy} - F1 Score: {f1} - ROC AUC: {roc_auc} - KNN: {grid_search_knn.best_params_}')

Fitting 5 folds for each of 162 candidates, totalling 810 fits
Accuracy: 0.9365079365079365 - F1 Score: 0.9459459459459459 - ROC AUC: 0.9405263157894735 - Random Forest: {'bootstrap': True, 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Fitting 5 folds for each of 56 candidates, totalling 280 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Accuracy: 0.9206349206349206 - F1 Score: 0.9411764705882353 - ROC AUC: 0.9444444444444444 - KNN: {'metric': 'chebyshev', 'n_neighbors': 3, 'weights': 'uniform'}


Dada la cantidad de datos de entrenamiento, se utilizaran como `n_estimators=[50, 100, 200]` (n* arboles) y `max_depth=[10, 20, 30, 40, 50]` (profundidad del arbol) para el algoritmo de RandomForest, ya que esta combinacion hace que el modelo no sea tan costoso computacionalmente y tenga un buen rendimiento. Tambien, como `min_samples_split=[2, 5, 10]` y `min_samples_leaf=1` para que el modelo no sea tan restrictivo (con valores no tan grandes) y evite el overfitting. Y, por ultimo, `bootsrap=[1, 2, 4]` para que el modelo sea aleatorio y no se sobreajuste a los datos.

Ambos modelos tienen un rendimiento similar, pero el modelo de Random Forest tiene un mejor rendimiento en términos de precisión y F1 Score. Por lo tanto, se puede utilizar el modelo de Random Forest para la clasificación de cultivos en la región de Egipto.